![imagenes](logo.png)

# Entrenamiento, prueba y validación

La práctica estándar en deep learning consiste en dividir el conjunto total de datos en dos subconjuntos:

**Conjunto de entrenamiento (training set):** es el subconjunto de datos que se utiliza para ajustar los parámetros de la red neuronal. Aquí es donde el cerebro aprende.

**Conjunto de prueba (test set):** es el subconjunto que se mantiene completamente separado y se utiliza exclusivamente para evaluar el rendimiento final del cerebro.

Esta separación debe hacerse antes de entrenar la red neuronal, y debe mantenerse inviolable: jamás se deben usar los datos de prueba para ajustar el modelo. Solo así podemos obtener una estimación honesta de su capacidad de aprendizaje.

<center>
  <img src="im04.png" style="width:500px; height:500px;">
</center>

## Entrenamiento

Durante el entrenamiento, el modelo utiliza los datos disponibles para ajustar sus parámetros internos. En una red neuronal, significa ajustar los pesos y sesgos mediante retropropagación. El objetivo es reducir el error de predicción en los datos conocidos.

Pero aquí está la trampa: el objetivo del entrenamiento no es únicamente predecir bien los datos de entrenamiento, sino predecir bien datos que el modelo nunca ha visto. Entrenar bien no garantiza aprender bien.

La división típica es entre un 70% u 80% para entrenamiento, y un 30% o 20% para prueba. Esta proporción puede variar dependiendo del tamaño del conjunto total. 

Si se dispone de muy pocos datos, usar una sola partición puede ser arriesgado. En esos casos, se recomienda usar técnicas como la **validación cruzada**, donde el conjunto de prueba rota, o el **bootstrap**, que crea muestras con reemplazo.

## Validación cruzada

La validación cruzada (cross-validation) en DL es una técnica de evaluación de modelos que permite estimar su capacidad de generalización, especialmente útil cuando se tienen conjuntos de datos limitados. Se aplica con adaptaciones debido al alto coste computacional.

La idea principal es dividir los datos en $K$ particiones (folds), entrenar el modelo $K$ veces usando $K-1$ particiones como entrenamiento y 1 partición como validación, rotando la partición de validación cada vez.


<center>
  <img src="im05.png">
</center>

## Proceso de la validación cruzada

Supongamos una arquitectura general $n_0-n_1-n_2-...-n_R-n_{out}$ con $p$ parámetros (ya sabemos cómo calcular $p$). Tomemos


- $d$ datos en total (es decir, $d$ clientes).
- Tamaño de batch $B$ (recuerda que $B=d$ es gradiente descendente; $B=1$ es gradiente estocástico y $1<B<d$ es minibatch).
- $E$ épocas
- Separación en entrenamiento y prueba de $0<e<1$ para entrenamiento (y $1-e$ para prueba)
- $K$ particiones (folds) para la validación cruzada.

### 1. Separación estructural inicial

Separamos en entrenamiento y prueba: $\mathcal{D}_{train}$, con $|\mathcal{D}_{train}|=e\cdot d$, y $\mathcal{D}_{test}$, con $|\mathcal{D}_{test}|=(1-e)\cdot d$. El conjunto $\mathcal{D}_{test}$ **no vuelve a tocarse**.

### 2. Construcción de los folds

Partimos $\mathcal{D}_{train}$ en $K$ folds:
$$\mathcal{D}_{train}=P_1\cup P_2\cup P_3\cup...\cup P_K\mbox{ con }|P_k|=\frac{e\cdot d}{K}$$

Cada $P_k$ tiene el $\frac{100}{K}\%$ de la tabla de entrenamiento.

### 3. Ciclo de la validación cruzada

Para cada $k\in\{1,2,...,K\}$ se definen:
- Entrenamiento del fold:
 $$\mathcal{D}_{train}^{(k)}=\mathcal{D}_{train}\backslash P_k$$
- Validación del fold:
 $$\mathcal{D}_{val}^{(k)}=P_k$$

### 4. ENTRENAMIENTO

Para cada época $e\in\{1,2,...,E\}$:
1. Se reordena $\mathcal{D}_{train}^{(k)}$
2. Se parte en minibatches de tamaño $B$
3. Para cada minibatch
   - Propagación hacia adelante
   - Cálculo de la función de pérdida
   - Propagación hacia atrás
   - Actualización de los $p$ parámetros

### 5. Evaluación del fold

Al terminar las $E$ épocas, se calcula la **pérdida de validación** $$\mathcal{L}_{val}^{(k)}=\mathcal{L}(\mathcal{D}_{val}^{(k)})$$

### 6. Resultado de la validación cruzada

Después de los $K$ folds: $$\mathcal{L}_{CV}=\frac{1}{K}\sum_{k=1}^K\mathcal{L}_{val}^{(k)}$$

Este valor se usa para comparar arquitecturas, regularización, elegir hiperparámetros ($\eta$, $B$, $E$) o detectar sobreajuste.

### 7. Paso final (fuera de la CV)

Una vez tomada la decisión 
1. Se reentrena la red desde cero usando todo $\mathcal{D}_{train}$
2. Se evalúa una sola vez en $\mathcal{D}_{test}$ y se obtiene $\mathcal{L}_{test}$